In [1]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [2]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

**Activate Webdriver**

In [3]:
# initiate a webdriver in selenium 
# since website data is dynamically generated

driver = activate_web_driver('firefox')

2022-12-14 08:01:03,936 INFO: ====== WebDriver manager ======
2022-12-14 08:01:04,381 INFO: Get LATEST geckodriver version for 107.0 firefox


[WDM] - Downloading: 19.0kB [00:00, 9.74MB/s]                   


2022-12-14 08:01:04,975 INFO: Getting latest mozilla release info for v0.32.0
2022-12-14 08:01:05,265 INFO: About to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.32.0/geckodriver-v0.32.0-win64.zip


[WDM] - Downloading: 100%|██████████| 1.58M/1.58M [00:00<00:00, 7.09MB/s]


2022-12-14 08:01:06,276 INFO: Driver has been saved in cache [C:\Users\Chris\.wdm\drivers\geckodriver\win64\0.32]


**Scrape New Completed Games and Format**

In [4]:

def get_new_games(driver)-> pd.DataFrame:

    # set search for yesterday's games
    DAYS = 1
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")


    df = scrape_to_dataframe(driver, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    df = convert_columns(df)

    print(df.info())
    df = combine_home_visitor(df)

    return df

df_new = get_new_games(driver)

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
SEASON = df_new['SEASON'].max()

df_new




<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   HOME            10 non-null     int64         
 1   GAME_DATE_EST   10 non-null     datetime64[ns]
 2   HOME_TEAM_WINS  10 non-null     int64         
 3   PTS             10 non-null     int64         
 4   FG_PCT          10 non-null     float64       
 5   FG3_PCT         10 non-null     float64       
 6   FT_PCT          10 non-null     float64       
 7   REB             10 non-null     int64         
 8   AST             10 non-null     int64         
 9   TEAM_ID         10 non-null     object        
 10  GAME_ID         10 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 960.0+ bytes
None


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2022-12-13,0,103,41.3,23.8,85.0,48,21,1610612758,22200409,123,51.2,45.7,77.8,42,34,1610612755,2022
1,2022-12-13,0,111,40.0,40.0,78.9,37,25,1610612744,22200410,128,54.9,41.4,81.3,55,22,1610612749,2022
2,2022-12-13,0,97,32.0,25.5,94.7,44,22,1610612756,22200411,111,42.0,34.3,91.2,67,16,1610612745,2022
3,2022-12-13,0,100,39.8,14.8,66.7,43,25,1610612740,22200412,121,47.6,31.9,66.7,59,31,1610612762,2022
4,2022-12-13,1,122,44.2,39.6,68.8,51,27,1610612738,22200413,118,47.3,27.6,73.3,49,23,1610612747,2022


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
teams_list = get_todays_matchups(driver)

# example output:
# ['/team/1610612759/spurs/', '/team/1610612748/heat/',...

print(teams_list)

# create list of matchups by parsing out team ids from teams_list
# second team id is always the home team
team_count = len(teams_list) 
matchups = []
for i in range(0,team_count,2):
    visitor_id = teams_list[i].partition("team/")[2].partition("/")[0] #extract team id from text
    home_id = teams_list[i+1].partition("team/")[2].partition("/")[0]
    matchups.append([visitor_id, home_id])

matchups

['/team/1610612758/kings/', '/team/1610612755/sixers/', '/team/1610612744/warriors/', '/team/1610612749/bucks/', '/team/1610612756/suns/', '/team/1610612745/rockets/', '/team/1610612740/pelicans/', '/team/1610612762/jazz/', '/team/1610612738/celtics/', '/team/1610612747/lakers/']


[['1610612758', '1610612755'],
 ['1610612744', '1610612749'],
 ['1610612756', '1610612745'],
 ['1610612740', '1610612762'],
 ['1610612738', '1610612747']]

**Close Webdriver**

In [6]:
driver.close() 

**Create Blank Rows**

In [8]:
# append today's matchups to the new games dataframe

# since we don't have access to official game ids, we will use the 20000001 + index as a game id
# 20000001 is used to insure that feature engineering will treat this as a regular season game
# this data will be updated later after the game is played


df_today = pd.DataFrame()
for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 
                    'VISITOR_TEAM_ID': matchup[0], 
                    'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                    'GAME_ID': 20000001+i,
                    'SEASON': SEASON,
                    } 
    game_details_df = pd.DataFrame(game_details, index=[i])
    # append to new games dataframe
    df_today = pd.concat([df_today, game_details_df], ignore_index = True)

#blank rows will be filled with 0 to prevent issues with feature engineering
df_today = df_today.fillna(0) 

df_today


,HOME_TEAM_ID,VISITOR_TEAM_ID,GAME_DATE_EST,GAME_ID,SEASON
0,1610612755,1610612758,2022-12-14,20000001,2022
1,1610612749,1610612744,2022-12-14,20000002,2022
2,1610612745,1610612756,2022-12-14,20000003,2022
3,1610612762,1610612740,2022-12-14,20000004,2022
4,1610612747,1610612738,2022-12-14,20000005,2022


**Access Feature Store**

In [9]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350


Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [10]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [11]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


2022-12-14 08:08:52,779 INFO: USE `nba_predictor_featurestore`
2022-12-14 08:08:53,103 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_1` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2018-12-10,21800395,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,2007-03-12,20600946,1610612756,1610612745,2006,103,0.500000,0.727051,0.600098,18,50,82,0.385986,0.722168,0.262939,13,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22607,2022-12-11,20000003,1610612752,1610612758,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
22608,2022-12-10,22200394,1610612750,1610612757,2022,118,54.187500,75.000000,43.500000,22,28,124,51.812500,89.312500,38.187500,20,45,0
22609,2022-12-10,22200389,1610612746,1610612764,2022,114,42.593750,93.812500,39.593750,22,42,107,43.500000,89.500000,47.093750,24,50,1
22610,2022-12-11,20000005,1610612755,1610612766,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0


**Convert Feature Names back to original mixed case**

In [12]:
df_old = convert_feature_names(df_old)
df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2018-12-10,21800395,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,2007-03-12,20600946,1610612756,1610612745,2006,103,0.500000,0.727051,0.600098,18,50,82,0.385986,0.722168,0.262939,13,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22607,2022-12-11,20000003,1610612752,1610612758,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
22608,2022-12-10,22200394,1610612750,1610612757,2022,118,54.187500,75.000000,43.500000,22,28,124,51.812500,89.312500,38.187500,20,45,0
22609,2022-12-10,22200389,1610612746,1610612764,2022,114,42.593750,93.812500,39.593750,22,42,107,43.500000,89.500000,47.093750,24,50,1
22610,2022-12-11,20000005,1610612755,1610612766,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0


**Update Yesterday's Matchup Predictions with New Final Results**

In [19]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results
#>>> df1.set_index('Code', inplace=True)
#>>> df1.update(df2.set_index('Code'))
#>>> df1.reset_index()  # to recover the initial structure

df_pending = df_old[df_old['GAME_ID'] < 20000100]

df_pending.update(df_new)

df_pending

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
22603,2022-12-11,20000006,1610612737,1610612741,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
22604,2022-12-11,20000004,1610612753,1610612761,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
22605,2022-12-11,20000001,1610612740,1610612756,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
22606,2022-12-11,20000002,1610612765,1610612747,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
22607,2022-12-11,20000003,1610612752,1610612758,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
22610,2022-12-11,20000005,1610612755,1610612766,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
22611,2022-12-11,20000007,1610612745,1610612749,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0


**Add Today's Matchups for Feature Engineering**

In [14]:
df_combined = pd.concat([df_combined, df_today], ignore_index = True)
df_combined

,HOME_TEAM_ID,VISITOR_TEAM_ID,GAME_DATE_EST,GAME_ID,SEASON
0,1610612755,1610612758,2022-12-14,20000001,2022
1,1610612749,1610612744,2022-12-14,20000002,2022
2,1610612745,1610612756,2022-12-14,20000003,2022
3,1610612762,1610612740,2022-12-14,20000004,2022
4,1610612747,1610612738,2022-12-14,20000005,2022


**Data Processing**

In [15]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

KeyError: 'HOME_TEAM_WINS'

**Feature Engineering**

In [ ]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined


**Insert New Data into Feature Group**

In [ ]:

def test():
    # retrieve only new games from the combined dataframe now that feature engineering is complete

    # set index to GAME_ID
    df_combined = df_combined.set_index('GAME_ID')
    df_new = df_new.set_index('GAME_ID')
    
    # retrieve only new games
    df_new = df_combined.loc[df_new.index]

    # reset GAME_ID index back to column
    df_new = df_new.reset_index()

    # convert certain features back to int32 for Hopsworks compatibility
    df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')
    df_new['HOME_TEAM_WINS'] = df_new['HOME_TEAM_WINS'].astype('int32')
    df_new['TARGET'] = df_new['TARGET'].astype('int32')

    # save new games to Hopsworks feature group
    rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

    df_new 

rolling_stats_fg.insert(df_combined, write_options={"wait_for_job" : False})